## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.
C:\Users\LJY\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\LJY\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\LJY\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\LJY\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [32]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
classifier.add(Convolution2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
classifier.add(MaxPooling2D(pool_size=2))

classifier.add(Convolution2D(filters=128, kernel_size=3, activation='relu', padding='same'))
classifier.add(Convolution2D(filters=128, kernel_size=3, activation='relu', padding='same'))
classifier.add(MaxPooling2D(pool_size=2))

classifier.add(Convolution2D(filters=128, kernel_size=3, activation='relu', padding='same'))
classifier.add(Convolution2D(filters=128, kernel_size=3, activation='relu', padding='same'))
classifier.add(MaxPooling2D(pool_size=2))


#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=64,epochs=13)


C:\Users\LJY\anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\LJY\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/13
50000/50000 [==============================] - 304s 6ms/step - loss: 1.3360 - accuracy: 0.5174
Epoch 2/13
50000/50000 [==============================] - 327s 7ms/step - loss: 0.8169 - accuracy: 0.7139
Epoch 3/13
50000/50000 [==============================] - 326s 7ms/step - loss: 0.6309 - accuracy: 0.7817
Epoch 4/13
50000/50000 [==============================] - 329s 7ms/step - loss: 0.5026 - accuracy: 0.8239
Epoch 5/13
50000/50000 [==============================] - 315s 6ms/step - loss: 0.4069 - accuracy: 0.8572
Epoch 6/13
50000/50000 [==============================] - 327s 7ms/step - loss: 0.3211 - accuracy: 0.8876
Epoch 7/13
50000/50000 [==============================] - 400s 8ms/step - loss: 0.2589 - accuracy: 0.9085
Epoch 8/13
50000/50000 [==============================] - 429s 9ms/step - loss: 0.2118 - accuracy: 0.9253
Epoch 9/13
50000/50000 [==============================] - 426s 9ms/step - loss: 0.1855 - accuracy: 0.9345
Epoch 10/13
50000/50000 [=====================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [15]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.5580288e-04, 3.8260605e-06, 1.2015704e-01, 8.2388383e-01,
        5.1848073e-02, 2.4389809e-04, 2.9768760e-04, 2.9989465e-05,
        3.1799313e-03, 6.2258483e-09]], dtype=float32)

In [33]:
scores = classifier.evaluate(x_test, y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 19s 2ms/step
Test loss: 0.9950419802904129
Test accuracy: 0.7871000170707703
